In [1]:
# notebook settings
%load_ext autoreload
%autoreload 2

# external imports
import numpy as np
import matplotlib.pyplot as plt

# internal imports
from controller import controller
from mld_dynamics import mld, h, x_min, x_max, fc_min, fc_max, d, l

Using license file /Users/tobia/codes/gurobi.lic
Academic license - for non-commercial use only
Time horizon: 34. Convergence index: 0.000000. Number of facets: 162.
Maximal constraint-admissible invariant set found. Removing redundant facets ... minimal facets are 102.


## Statistical Analysis

In [2]:
# initial push towards the right wall
x0 = np.array([0., 0., 1., 0.])

e_sd = 0.# 0.001, 0.003, 0.01
N_sim = 50
N_samples = 1
i_0 = 0

nodes_keys = ['cs', 'ws', 'len_ws', 'grb', 'grb_fair']
times_keys = ['cs', 'ws', 'ws_constr', 'grb', 'grb_fair']

if i_0 == 0:
    nodes = {k: [] for k in nodes_keys}
    times = {k: [] for k in times_keys}
    errors = []
else:
    nodes = {k: list(np.load('data/nodes_'+k+'_sd_{:.3f}.npy'.format(e_sd))) for k in nodes_keys}
    times = {k: list(np.load('data/times_'+k+'_sd_{:.3f}.npy'.format(e_sd))) for k in times_keys}
    errors = list(np.load('data/errors_sd_{:.3f}.npy'.format(e_sd)))
    

with open('data/solve_log_sd_{:.3f}.log'.format(e_sd), 'w') as f:

    f.write('Error standard deviation {:.3f}\n\n'.format(e_sd)) 
    f.flush()

    i = i_0

    while len(nodes['cs']) < N_samples:
        np.random.seed(i)
        
        rollout = len(nodes['cs'])
        f.write(f'\nRollout {rollout}\n')
        f.write(f'Iteration n.{i}\n\n')
        f.flush()
        rollout_success = True

        nodes_i = {k: [] for k in nodes_keys}
        times_i = {k: [] for k in times_keys}
        errors_i = []

        x_sim = [x0]
        ws = None

        for t in range(N_sim):
            print((i,t), end='\r')
            f.write('Time step %d '%t)
            f.flush()

#             # project previous state in feasible set
#             x_prev = np.minimum(np.maximum(x_sim[-1], x_min), x_max)
                
            # solve without warm start
            try:
                solution_cs, leaves_cs, n_nodes_cs, time_cs = controller.feedforward(
                    x_sim[-1],
                    printing_period=None,
                    gurobi_options={'Method': 1} # dual simplex
                )
            except:
                rollout_success = False
                f.write('\nUnseccessful rollout, cold-started BB broke.')
                break
            nodes_i['cs'].append(n_nodes_cs)
            times_i['cs'].append(time_cs)
            f.write('(cs: {}, {:.3f}) '.format(n_nodes_cs, time_cs))
            f.flush()

            # solve with warm start
            try:
                solution_ws, leaves_ws, n_nodes_ws, time_ws = controller.feedforward(
                    x_sim[-1],
                    warm_start=ws,
                    printing_period=None,
                    gurobi_options={'Method': 1} # dual simplex
                )
            except:
                rollout_success = False
                f.write('\nUnseccessful rollout, warm-started BB broke.')
                break
            nodes_i['ws'].append(n_nodes_ws)
            times_i['ws'].append(time_ws)
            f.write('(ws: {}, {:.3f}) '.format(n_nodes_ws, time_ws))
            f.flush()
            
            # solve with gurobi
            try:
                x_grb, cost_grb, n_nodes_grb, time_grb = controller.feedforward_gurobi(
                    x_sim[-1],
                    {'OutputFlag': 0, 'MIPGap': 0}
                )
            except:
                rollout_success = False
                f.write('\nUnseccessful rollout, Gurobi broke.')
                break
            nodes_i['grb'].append(n_nodes_grb)
            times_i['grb'].append(time_grb)
            f.write('(grb: {}, {:.3f}) '.format(n_nodes_grb, time_grb))
            f.flush()
            
            # solve with gurobi fair
            try:
                x_grb_fair, cost_grb_fair, n_nodes_grb_fair, time_grb_fair = controller.feedforward_gurobi(
                    x_sim[-1],
                    {'OutputFlag': 0, 'MIPGap': 0, 'Presolve': 0, 'Heuristics': 0, 'Threads': 1}
                )
            except:
                rollout_success = False
                f.write('\nUnseccessful rollout, Gurobi fair broke.')
                break
            nodes_i['grb_fair'].append(n_nodes_grb_fair)
            times_i['grb_fair'].append(time_grb_fair)
            f.write('(grb_fair: {}, {:.3f}) '.format(n_nodes_grb_fair, time_grb_fair))
            f.flush()

            # check feasibility
            if solution_cs is None:
                break
            assert np.isclose(solution_cs.objective, solution_ws.objective)
            assert np.isclose(solution_cs.objective, cost_grb)
            assert np.isclose(solution_cs.objective, cost_grb_fair)

            # generate random error of specified norm
            e_t = e_sd * np.multiply(np.random.randn(mld.nx), x_max)
            errors_i.append(e_t)

            # generate warm start
            ws, time_ws_constr = controller.construct_warm_start(
                leaves_ws,
                solution_ws.variables['x'][0],
                solution_cs.variables['uc'][0],
                solution_cs.variables['ub'][0],
                e_t
            )
            nodes_i['len_ws'].append(len(ws))
            times_i['ws_constr'].append(time_ws_constr)
            f.write('(ws info: {}, {:.3f}) '.format(len(ws), time_ws_constr))
            f.flush()

            # update state
            x_sim.append(solution_ws.variables['x'][1] + e_t)
            f.write('(e: {:.3f}, {})\n'.format(np.linalg.norm(e_t), e_t))
            f.flush()

        if rollout_success:
            
            for k in nodes_keys:
                nodes[k].append(nodes_i[k])
                np.save('data/nodes_'+k+'_sd_{:.3f}.npy'.format(e_sd), nodes[k])
            for k in times_keys:
                times[k].append(times_i[k])
                np.save('data/times_'+k+'_sd_{:.3f}.npy'.format(e_sd), times[k])
            errors.append(errors_i)
            np.save('data/errors_sd_{:.3f}.npy'.format(e_sd), errors)
#         else:
#             from controller import controller

        i += 1